# Create Test Dictionary

The purpose of this notebook is to create a dictionary of fuel moisture scenarios for model testing.

[USGS](https://water.usgs.gov/edu/activity-howmuchrain-metric.html#:~:text=Slight%20rain%3A%20Less%20than%200.5,than%208%20mm%20per%20hour.) classifies rainfall as:
* Slight: $\text{rain}<.5$ mm/hr
* Moderate: $.5<\text{rain}<4$ mm/hr
* Heavy: $4<\text{rain}<8$ mm/hr
* Very Heavy: $8<\text{rain}$ mm/hr

## Setup

In [ ]:
# Change path for module imports
import os
os.chdir('..')

import numpy as np, random
from numpy.random import rand
import matplotlib.pyplot as plt
from data_funcs import synthetic_data, plot_data, to_json, from_json, format_raws, mse_data

import moisture_models as mod
from moisture_models import model_decay, model_moisture
from datetime import datetime, timedelta
import json
from utils import hash2

from MesoPy import Meso
meso_token="4192c18707b848299783d59a9317c6e1"
m=Meso(meso_token)

import reproducibility

In [ ]:
# Change directory to data
os.chdir('data')

---

**Scenarios 1-6**: simulated moisture with default equilibrium parameters. Vary rain from none up to very heavy as described above.

In [ ]:
# Dictionary to be saved for testing
test_dict = {}

In [ ]:
## Case 1: no rain
synt_dat=synthetic_data(max_rain = 0) 
synt_dat['id'] = 'case1'
synt_dat['title'] = 'Synthetic Data (no rain)'
synt_dat['descr'] = 'Max rain: 0 mm/hr; Emin: 5; Emax: 30; 20 days'

test_dict['case1'] = synt_dat # save to test dictionary

In [ ]:
## Case 2: light rain
synt_dat=synthetic_data(max_rain = 0.4) 
synt_dat['id'] = 'case2'
synt_dat['title'] = 'Synthetic Data (light rain)'
synt_dat['descr'] = 'Max rain: .4 mm/hr; Emin: 5; Emax: 30; 20 days'

test_dict['case2'] = synt_dat # save to test dictionary

In [ ]:
## Case 3: moderate rain
synt_dat=synthetic_data(max_rain = 3) 
synt_dat['id'] = 'case3'
synt_dat['title'] = 'Synthetic Data (med. rain)'
synt_dat['descr'] = 'Max rain: 3 mm/hr; Emin: 5; Emax: 30; 20 days'

test_dict['case3'] = synt_dat # save to test dictionary

In [ ]:
## Case 4: heavy rain
synt_dat=synthetic_data(max_rain = 6) 
synt_dat['id'] = 'case4'
synt_dat['title'] = 'Synthetic Data (heavy rain)'
synt_dat['descr'] = 'Max rain: 6 mm/hr; Emin: 5; Emax: 30; 20 days'

test_dict['case4'] = synt_dat # save to test dictionary

In [ ]:
## Case 5: very heavy rain 1
synt_dat=synthetic_data(max_rain = 10) 
synt_dat['id'] = 'case5'
synt_dat['title'] = 'Synthetic Data (very heavy rain 1)'
synt_dat['descr'] = 'Max rain: 10 mm/hr; Emin: 5; Emax: 30; 20 days'

test_dict['case5'] = synt_dat # save to test dictionary

In [ ]:
## Case 6: very heavy rain 2
synt_dat=synthetic_data(max_rain = 15) 
synt_dat['id'] = 'case6'
synt_dat['title'] = 'Synthetic Data (very heavy rain 2)'
synt_dat['descr'] = 'Max rain: 15 mm/hr; Emin: 5; Emax: 30; 20 days'

test_dict['case6'] = synt_dat # save to test dictionary

In [ ]:
plot_data(test_dict['case6'])

---

**Scenario 7-8:** RAWS Data, multiple time slices

In [ ]:
## Read RAWS data with MesoPy

time_start = "201806010800"  # June 1 2018 08:00 in format yyyymmddHHMM
time_end   = "201907200900"  # June 20 2018 09:00 in format yyyymmddHHMM

vars='air_temp,relative_humidity,precip_accum,fuel_moisture'

In [ ]:
meso_ts = m.timeseries(time_start, time_end, stid="CPTC2", showemptystations = '0', vars=vars)   # ask the object for data

In [ ]:
raws1 = format_raws(meso_ts['STATION'][0])

In [ ]:
# Scenario 7: time 0-1200 for station
## Heavy rain at end of time period
dict1={
    'id': 'case7',
    'time': raws1['time'][0:1200],
    'rain': raws1['rain'][0:1200],
    'fm' : raws1['fm'][0:1200],
    'rh' : raws1['rh'][0:1200],
    'temp' : raws1['temp'][0:1200],
    'Ed' : raws1['Ed'][0:1200],
    'Ew' : raws1['Ew'][0:1200],
    'STID' : raws1['STID'],
    'title' : 'RAWS Station CPTC2 #1',
    'descr' : 'Real surface level data, very heavy rain at end',
    'hours':1200,
    'h2':300,
    'other': {'lon': raws1['lon'], 'lat': raws1['lat']}
}

test_dict['case7'] = dict1 # save to test dictionary

In [ ]:
plot_data(dict1)

In [ ]:
# Scenario 8: time 800-2000 for station
## Heavy rain at beginning of time period
dict1={
    'id': 'case8',
    'time': raws1['time'][800:2000],
    'rain': raws1['rain'][800:2000],
    'fm' : raws1['fm'][800:2000],
    'rh' : raws1['rh'][800:2000],
    'temp' : raws1['temp'][800:2000],
    'Ed' : raws1['Ed'][800:2000],
    'Ew' : raws1['Ew'][800:2000],
    'STID' : raws1['STID'],
    'title' : 'RAWS Station CPTC2 #2',
    'descr' : 'Real surface level data, very heavy rain at beginning',
    'hours':1200,
    'h2':300,
    'other': {'lon': raws1['lon'], 'lat': raws1['lat']}
}

test_dict['case8'] = dict1 # save to test dictionary

In [ ]:
plot_data(test_dict['case8'])

---

**Scenario 9-10:** RTMA Data, multiple time slices at station BKCU1

In [ ]:
rtma = from_json('rtma.json')

In [ ]:
## Read RAWS data with MesoPy

time_start = "201807041600"  # '2018-07-04 16:00'
time_end   = "201810040900"  # '2018-10-04 08:00', 1 hr buffer

vars='fuel_moisture'

meso_ts = m.timeseries(time_start, time_end, stid="BKCU1", showemptystations = '0', vars=vars)   # ask the object for data

In [ ]:
from data_funcs import format_rtma

In [ ]:
rtma1 = format_rtma(rtma)
fm = np.array(meso_ts['STATION'][0]['OBSERVATIONS']['fuel_moisture_set_1'])

In [ ]:
# Scenario 9: time 800:2000 for rtma location
## moderate rain towards end of time period
dict1={
    'id': 'case9',
    'time': rtma1['time'][800:2000],
    'rain': rtma1['rain'][800:2000],
    'fm' : fm[0:1200],
    'rh' : rtma1['rh'][800:2000],
    'temp' : rtma1['temp'][800:2000],
    'Ed' : rtma1['Ed'][800:2000],
    'Ew' : rtma1['Ew'][800:2000],
    'title' : 'RTMA Data, Fuel from RAWS Station BKCU1 #1',
    'descr' : 'rtma weather, surface level fuel, moderate rain at end',
    'hours':1200,
    'h2':300,
    'other': {'lon': rtma1['lon'], 'lat': rtma1['lat']}
}

test_dict['case9'] = dict1 # save to test dictionary

In [ ]:
plot_data(test_dict['case9'])

In [ ]:
# Scenario 10: time 1000-2200 for same location
## moderate rain towards end of time period
dict1={
    'id': 'case10',
    'time': rtma1['time'][1800:3000],
    'rain': rtma1['rain'][1800:3000],
    'fm' : fm[1000:2200],
    'rh' : rtma1['rh'][1800:3000],
    'temp' : rtma1['temp'][1800:3000],
    'Ed' : rtma1['Ed'][1800:3000],
    'Ew' : rtma1['Ew'][1800:3000],
    'title' : 'RTMA Data, Fuel from RAWS Station BKCU1 #2',
    'descr' : 'rtma weather, surface level fuel, heavy rain at end',
    'hours':1200,
    'h2':300,
    'other': {'lon': rtma1['lon'], 'lat': rtma1['lat']}
}

test_dict['case10'] = dict1 # save to test dictionary

In [ ]:
plot_data(test_dict['case10'])

### Scenario 11: Original RNN

RNN generated from notebook to a json file, used for reproducibility since the beginning. RTMA data from same station.

In [ ]:
rnn = from_json('rnn_orig.json')

In [ ]:
rnn.keys()

In [ ]:
# Scenario 11:
N = rnn['Ed'].shape[0]

dict1={
    'id': 'case11',
    'time': None,
    'rain': rnn['rain'][0:N],
    'fm' : rnn['fm'][0:N],
    'Ed' : rnn['Ed'][0:N],
    'Ew' : rnn['Ew'][0:N],
    'rain' : rnn['rain'][0:N],
    'title' : 'RNN Orig',
    'descr' : 'rtma weather, surface level fuel, moderate rain at end',
    'hours':N,
    'h2':300,
    'other': {'lon': rtma1['lon'], 'lat': rtma1['lat']}
}

test_dict['case11'] = dict1 # save to test dictionary

In [ ]:
plot_data(test_dict['case11'])

## Save Output

In [ ]:
import pickle
with open('testing_dict.pickle', 'wb') as handle:
    pickle.dump(test_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Sample KF Loop on Dict

In [ ]:
with open('testing_dict.pickle', 'rb') as handle:
    test_dict = pickle.load(handle)

In [ ]:
err_dict = {} # dictionary to save validation error for cases

In [ ]:
%%capture cap --no-stderr

# Loop through Test dictionary and run model
# Save Print output as text file for quick reference
with open("errors_KF.txt", "a") as f:
    for key in [*test_dict.keys()]:
        print(key, ':', test_dict[key]['title'])

        dict1 = test_dict[key]
        m,Ec = mod.run_augmented_kf(dict1)
        dict1['m']=m

        errs = mse_data(dict1)

        print('-'*25)

        err_dict[key] = {
            'title' : test_dict[key]['title'],
            'train' : errs[0],
            'test' : errs[1],
            'model_type' : 'augmented KF',
            'm_hash' : int(hash2(m)),
            'm' : m,
            'Ec' : Ec # equil. correction learned by training
        }

In [ ]:
# Print above output for visualization, save to text file

print(cap.stdout)

with open('errors_KF.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
## Save Validation Outputs as pickle file with all data

with open('errors_KF.pickle', 'wb') as handle:
    pickle.dump(err_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)